In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Cystic_Fibrosis/GSE35713'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional Signatures as a Disease-Specific and Predictive Inflammatory Biomarker for Type 1 Diabetes"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['responder cell line: UPN727', 'protocol: stimulated with Auto250S7 plasma', 'protocol: stimulated with HC2074 plasma', 'protocol: stimulated with HC2302 plasma', 'protocol: stimulated with HC2593 plasma', 'protocol: stimulated with RO1973 plasma', 'protocol: stimulated with RO2130 plasma', 'protocol: stimulated with RO2312 plasma', 'protocol: stimulated with RO1964 plasma', 'protocol: stimulated with RO2348 plasma', 'protocol: stimulated with RO2366 plasma', 'protocol: stimulated with RO2598 plasma', 'protocol: stimulated with Auto250S8 plasma', 'protocol: stimulated with HC2146 plasma', 'protocol: stimulated with HC223 plasma', 'protocol: stimulated with RO1939 plasma', 'p

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
is_gene_available = True  # Based on STEP 1 output, assuming gene expression data is included since no exclusion criteria met

# Data Availability Check
sample_characteristics = {
    0: ['responder cell line: UPN727', 'protocol: stimulated with Auto250S7 plasma', 'protocol: stimulated with HC2074 plasma', 
        'protocol: stimulated with HC2302 plasma', 'protocol: stimulated with HC2593 plasma', 'protocol: stimulated with RO1973 plasma', 
        'protocol: stimulated with RO2130 plasma', 'protocol: stimulated with RO2312 plasma', 'protocol: stimulated with RO1964 plasma', 
        'protocol: stimulated with RO2348 plasma', 'protocol: stimulated with RO2366 plasma', 'protocol: stimulated with RO2598 plasma', 
        'protocol: stimulated with Auto250S8 plasma', 'protocol: stimulated with HC2146 plasma', 'protocol: stimulated with HC223 plasma', 
        'protocol: stimulated with RO1939 plasma', 'protocol: stimulated with RO2109 plasma', 'protocol: stimulated with Auto250S9 plasma', 
        'protocol: stimulated with HC285 plasma', 'protocol: stimulated with HC233 plasma', 'protocol: stimulated with HC294 plasma', 
        'protocol: stimulated with HC224 plasma', 'protocol: stimulated with RO253 plasma', 'protocol: stimulated with RO2024 plasma', 
        'protocol: stimulated with RO1037 plasma', 'protocol: stimulated with RO1437 plasma', 'protocol: stimulated with RO2944 plasma', 
        'protocol: stimulated with RO2953 plasma', 'protocol: stimulated with Auto250S10 plasma', 'protocol: stimulated with HC217 plasma'], 
    1: ['protocol: stimuated with AutoB1S1 plasma', 'protocol: stimuated with AutoB1S2 plasma', 'protocol: stimuated with AutoB1S3 plasma', 
        'protocol: stimuated with AutoB3W1 plasma', 'protocol: stimuated with AutoB3W1+2 plasma', 'protocol: stimulated with CF4017 plasma', 
        'protocol: stimulated with CF4024 plasma', 'protocol: stimulated with CF4026 plasma', 'protocol: stimulated with CF4045 plasma', 
        'protocol: stimulated with CF4047 plasma', 'protocol: stimulated with CF4087 plasma', 'protocol: stimulated with CF4091 plasma', 
        'protocol: stimulated with CF4107 plasma', 'protocol: stimulated with CF4088 plasma', 'protocol: stimulated with CF4133 plasma', 
        'protocol: stimulated with CF4138 plasma', 'protocol: stimulated with CF4157 plasma', 'protocol: stimulated with CF4184 plasma', 
        'protocol: stimulated with CF4256 plasma', 'protocol: stimulated with CF4265 plasma', 'protocol: stimulated with CF4271 plasma', 
        'protocol: stimulated with CF4297 plasma', 'protocol: stimulated with CF4304 plasma', 'protocol: stimulated with CF4305 plasma', 
        'protocol: stimulated with CF4285 plasma', 'protocol: stimulated with HC1271 plasma', 'protocol: stimulated with HC1365 plasma', 
        'protocol: stimulated with HC1301 plasma', 'protocol: stimulated with HC1249 plasma', 'protocol: stimulated with HC1265 plasma']
}

# Identify keys for 'Cystic_Fibrosis'
for key, values in sample_characteristics.items():
    if any('CF' in value for value in values):
        trait_row = key
        break

# Based on the given information, there's no age and gender directly indicated in the sample characteristics dictionary.
age_row = None
gender_row = None

# Define conversion functions
def convert_trait(value):
    if 'CF' in value:
        return 1
    else:
        return 0

def convert_age(value):
    try:
        return float(value.split(':')[1].strip())
    except ValueError:
        return None

def convert_gender(value):
    gender = value.split(':')[1].strip().lower()
    if gender == 'female':
        return 0
    elif gender == 'male':
        return 1
    else:
        return None

# Save cohort information
save_cohort_info('GSE35713', './preprocessed/Cystic_Fibrosis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Cystic_Fibrosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Cystic_Fibrosis/trait_data/GSE35713.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM873748': [0], 'GSM873749': [0], 'GSM873750': [0], 'GSM873751': [0], 'GSM873752': [0], 'GSM873753': [1], 'GSM873754': [1], 'GSM873755': [1], 'GSM873756': [1], 'GSM873758': [1], 'GSM873759': [1], 'GSM873760': [1], 'GSM873761': [1], 'GSM873762': [1], 'GSM873763': [1], 'GSM873764': [1], 'GSM873765': [1], 'GSM873766': [1], 'GSM873767': [1], 'GSM873768': [1], 'GSM873769': [1], 'GSM873770': [1], 'GSM873771': [1], 'GSM873772': [1], 'GSM873773': [1], 'GSM873774': [0], 'GSM873775': [0], 'GSM873776': [0], 'GSM873777': [0], 'GSM873778': [0], 'GSM873779': [0], 'GSM873780': [0], 'GSM873781': [0], 'GSM873782': [0], 'GSM873783': [0], 'GSM873784': [0], 'GSM873785': [0], 'GSM873786': [0], 'GSM873787': [0], 'GSM873788': [0], 'GSM873789': [0], 'GSM873790': [0], 'GSM873791': [0], 'GSM873792': [0], 'GSM873793': [0], 'GSM873794': [0], 'GSM873795': [0], 'GSM873796': [0], 'GSM873797': [0], 'GSM873800': [0], 'GSM873801': [0], 'GSM873802': [0], 'GSM873803': [0], 'GSM873804': [0], 'GSM873805': [0], 'GSM87380

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the keys for identifiers and gene symbols in the gene annotation dictionary
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(preview_df(gene_data))


{'GSM873748': [7.02143, 3.99158, 3.99158, 2.68914, 3.21356], 'GSM873749': [7.13402, 3.79929, 3.79929, 2.56659, 3.22859], 'GSM873750': [7.16147, 3.6665, 3.6665, 2.85174, 2.84358], 'GSM873751': [6.64351, 3.97479, 3.97479, 2.55753, 3.18672], 'GSM873752': [6.82251, 4.03708, 4.03708, 2.89281, 3.29246], 'GSM873753': [7.35671, 3.87646, 3.87646, 2.6447, 3.36356], 'GSM873754': [7.25638, 3.92406, 3.92406, 2.71165, 3.07434], 'GSM873755': [7.12382, 3.8876, 3.8876, 2.64236, 3.34501], 'GSM873756': [6.96944, 4.25138, 4.25138, 2.71493, 3.45668], 'GSM873758': [6.99481, 4.47985, 4.47985, 2.73072, 2.90601], 'GSM873759': [7.40452, 3.98811, 3.98811, 2.60028, 3.22297], 'GSM873760': [7.67058, 3.80658, 3.80658, 2.62301, 3.12619], 'GSM873761': [7.432, 3.89335, 3.89335, 2.76384, 3.0608], 'GSM873762': [7.21408, 4.46145, 4.46145, 2.69274, 3.48044], 'GSM873763': [7.40886, 3.97873, 3.97873, 2.59636, 3.01807], 'GSM873764': [7.31359, 3.83514, 3.83514, 2.60078, 3.48415], 'GSM873765': [7.65783, 4.00817, 4.00817, 2.5625

### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Cystic_Fibrosis/gene_data/GSE35713.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Cystic_Fibrosis')

# 4. Save the cohort information.
save_cohort_info('GSE35713', './preprocessed/Cystic_Fibrosis/cohort_info.json', True, True, trait_biased, merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Cystic_Fibrosis/GSE35713.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Cystic_Fibrosis', the least common label is '1.0' with 20 occurrences. This represents 9.85% of the dataset.
The distribution of the feature 'Cystic_Fibrosis' in this dataset is fine.

